# Imports

In [1]:
from time import time

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report

# Чтение и анализ датасета

In [2]:
df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


Дата сет представляет собой 50 000 пользовательских отзывов на 
фильмы с классификацией положительный `(positive)` или отрицательный `(negative)`

Посмотрим на таргет

In [4]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

Таргет сбалансирован, имеем по 25000 положительных и отрицательных отзывов

Сформируем пространство признаков и таргет.

In [5]:
X = df['review']
y = df['sentiment'].replace({'negative': 0, 'positive': 1})

Разобъем датасет на обучающую и тестовую выборки.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Bag of words (CountVectorizer)

## Обучение модели

Для классификации я выбрал `RandomForest`.

Но сначала нужно перевести текст отзывов в цифры. 
Для этого используем класс `CountVectorizer` который преобразовывает коллекцию текстовых 
документов в матрицу количества токенов, где каждый столбец соответствует отдельному 
токену (суть - слову в исходном тексте). 
Можно было бы показать данному векторайзеру все отзывы для того чтобы его словарь был полнее,
но думаю, что будет более честно и приближенно к реальности, когда не только сама модель,
но и различные трансформеры на этапе предпроцессинга тренируются только на обучающей выборке.

Параметр `min_df` указывает количество раз, которое слово должно встретиться в тексте, 
чтобы быть включенным в выходную матрицу.

Есть гипотеза, что эмоциональную окраску отзывам на тему хороший/плохой фильм
придают не так много слов и на обучающей выборке в 35000 отзывов они будут встречаться часто.
Поэтому можно попробовать установить это значение в широких пределах от 1 (взять все слова)
до 500 (взять слова, которые встретились не менее 500 раз). 
Посмотрим как это будет отражаться на точности классификации.

Метрики модели при различных значениях параметра `min_df` будем оценивать на кросс-валидации.
Натренированные на разных матрицах токенов векторайзеры будем сохранять в словаре `vectorizers`.

In [7]:
vectorizers = {}

model = RandomForestClassifier(random_state=42)

for mindf in [1, 50, 200, 500]:
    vectorizer = CountVectorizer(stop_words='english', min_df=mindf)
    X_train_transformed = vectorizer.fit_transform(X_train)
    vectorizers[mindf] = vectorizer
    print(f"min_df: {mindf}, X_train shape: {X_train_transformed.shape}")
    
    model_cv_score = np.mean(
        cross_val_score(model, X_train_transformed, y_train, cv=5, scoring="accuracy", n_jobs=-1)
    )
    print(f"Model cross validation score: {model_cv_score}\n")

min_df: 1, X_train shape: (35000, 87687)
Model cross validation score: 0.8553142857142857
min_df: 50, X_train shape: (35000, 7685)
Model cross validation score: 0.8473428571428571
min_df: 200, X_train shape: (35000, 2584)
Model cross validation score: 0.8372285714285713
min_df: 500, X_train shape: (35000, 1097)
Model cross validation score: 0.8287428571428572


В выводе видно, что при уменьшении размера словаря с 87687 слов до 1097,
точность модели незначительно падает с 85,5% до 82,8%

Возьмем для предсказания словарь, содержащий слова, которые встретились 
не менее 200 раз и натренируем модель на всей обучающей выборке

In [20]:
vectorizers

{1: CountVectorizer(stop_words='english'),
 50: CountVectorizer(min_df=50, stop_words='english'),
 200: CountVectorizer(min_df=200, stop_words='english'),
 500: CountVectorizer(min_df=500, stop_words='english')}

In [8]:
X_train_transformed = vectorizers[200].transform(X_train)

In [9]:
print(X_train_transformed)

  (0, 225)	1
  (0, 248)	1
  (0, 250)	1
  (0, 271)	2
  (0, 490)	1
  (0, 672)	1
  (0, 676)	1
  (0, 976)	1
  (0, 995)	1
  (0, 1032)	2
  (0, 1065)	1
  (0, 1102)	1
  (0, 1143)	1
  (0, 1155)	1
  (0, 1160)	1
  (0, 1349)	1
  (0, 1354)	1
  (0, 1386)	1
  (0, 1442)	1
  (0, 1513)	1
  (0, 1550)	1
  (0, 1552)	1
  (0, 1655)	1
  (0, 1695)	1
  (0, 1730)	1
  :	:
  (34999, 1093)	1
  (34999, 1110)	1
  (34999, 1196)	1
  (34999, 1233)	1
  (34999, 1354)	1
  (34999, 1380)	1
  (34999, 1404)	1
  (34999, 1412)	1
  (34999, 1435)	1
  (34999, 1513)	10
  (34999, 1678)	1
  (34999, 1742)	1
  (34999, 1793)	1
  (34999, 1825)	1
  (34999, 1840)	1
  (34999, 1947)	1
  (34999, 1989)	1
  (34999, 2038)	1
  (34999, 2176)	1
  (34999, 2179)	1
  (34999, 2258)	1
  (34999, 2289)	1
  (34999, 2329)	2
  (34999, 2356)	1
  (34999, 2396)	1


In [10]:
start_time = time()
model.fit(X_train_transformed, y_train)
print(f"Training time: {time() - start_time}")

Training time: 89.11080694198608


## Предсказание, вывод метрик

Перекодируем тестовую выборку, прогоним ее через модель, выведем метрики.

In [11]:
X_test_transformed = vectorizers[200].transform(X_test)

In [12]:
y_predicted = model.predict(X_test_transformed)
model_test_score = accuracy_score(y_true=y_test, y_pred=y_predicted)

print(f"Model test set score: {model_test_score}")
print("\n", classification_report(y_true=y_test, y_pred=y_predicted))

Model test set score: 0.8396

               precision    recall  f1-score   support

           0       0.83      0.85      0.84      7411
           1       0.85      0.83      0.84      7589

    accuracy                           0.84     15000
   macro avg       0.84      0.84      0.84     15000
weighted avg       0.84      0.84      0.84     15000


# TF - IDF (TfidfVectorizer)

TF - term frequency, IDF - inverse document frequency

Теперь попробуем подать на классификатор не количества найденных слов, а их TF-IDF индекс.

Делаем аналогичную последовательность действий:
- векторизация обучающей выборки;
- тренировка модели;
- векторизация тестовой выборки;
- проверка модели на тестовой выборке, вывод метрик.

In [13]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=200)

In [14]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [15]:
X_train_tfidf.shape

(35000, 2584)

In [16]:
print(X_train_tfidf)

  (0, 976)	0.12698426520305237
  (0, 2176)	0.08573780127745301
  (0, 2257)	0.139504208203309
  (0, 1354)	0.1035471930821437
  (0, 1032)	0.26363076998364265
  (0, 250)	0.22493038065566745
  (0, 1442)	0.195535607802665
  (0, 1655)	0.13341285107472725
  (0, 995)	0.07674092923257012
  (0, 2388)	0.16095587732147681
  (0, 225)	0.10292484921993565
  (0, 2382)	0.14444853821494988
  (0, 1552)	0.144381430048271
  (0, 1942)	0.13899011272372694
  (0, 1065)	0.13417883439884773
  (0, 271)	0.11986344681807265
  (0, 1143)	0.14301445638156335
  (0, 2577)	0.18754236658031506
  (0, 1550)	0.12080576671642523
  (0, 676)	0.21438503095934316
  (0, 2157)	0.2381165085168505
  (0, 1155)	0.1779034175048819
  (0, 248)	0.21479314385121762
  (0, 1102)	0.16834213330870518
  (0, 1349)	0.15229202432826924
  :	:
  (34999, 624)	0.13078230165397273
  (34999, 1196)	0.16843212627298232
  (34999, 1793)	0.1115578580581452
  (34999, 1989)	0.09848505968362213
  (34999, 233)	0.09102321791018046
  (34999, 1840)	0.102626969358202

In [17]:
start_time = time()
model.fit(X_train_tfidf, y_train)
print(f"Training time: {time() - start_time}")

Training time: 89.23653221130371


In [18]:
X_test_tfidf = vectorizer.transform(X_test)

In [19]:
y_predicted = model.predict(X_test_tfidf)
model_test_score = accuracy_score(y_true=y_test, y_pred=y_predicted)

print(f"Model test set score: {model_test_score}")
print("\n", classification_report(y_true=y_test, y_pred=y_predicted))

Model test set score: 0.8465333333333334

               precision    recall  f1-score   support

           0       0.84      0.86      0.85      7411
           1       0.86      0.84      0.85      7589

    accuracy                           0.85     15000
   macro avg       0.85      0.85      0.85     15000
weighted avg       0.85      0.85      0.85     15000


# Вывод

Выбор в качестве обучающих данных индексов TF-IDF вместо количественной матрицы
не отразился ни на времени тренировки модели, ни на ее точности.

В обоих случаях точность на тестовой выборке составила порядка 85%. 
На точность в бОльшей степени влияет размер словаря.